# CMS28 Smart Pix NN

## Library setup

Disable some console warnings

In [ ]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Import libraries

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
#import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Lambda
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import read_csv
import math
import seaborn as sns
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Input, Activation
from qkeras import *

%matplotlib inline
import matplotlib.pyplot as plt

Import plot `pT filter` function.

In [ ]:
from plot_pt_filter import plot_pt_filter
from plot_pt_filter import get_number_of_tracks

Define some additional helpers

In [ ]:
from IPython.display import display_html
from itertools import chain, cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h5 style="text-align: center;">{title}</h5>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [ ]:
def write_results(filename, model_id, loss, accuracy, GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0):
    import os
    import csv

    from datetime import datetime
    now = datetime.now()
    date_string = now.strftime("%d/%m/%Y %H:%M:%S")

    f = open(filename, 'a+')
    writer = csv.writer(f)
    if os.stat(filename).st_size == 0:
        writer.writerow(["date", "id", "loss", "accuracy", "nt_gev02", "nt_gev05", "nt_gev10", "nt_gev20"])
    writer.writerow([date_string, model_id, loss, accuracy, GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0])
    f.close()
    
def write_hw_results(filename, model_id, hls_area, ls_area, latency):
    import os
    import csv

    from datetime import datetime
    now = datetime.now()
    date_string = now.strftime("%d/%m/%Y %H:%M:%S")

    f = open(filename, 'a+')
    writer = csv.writer(f)
    if os.stat(filename).st_size == 0:
        writer.writerow(["date", "id", "HLS area", "LS area", "latency"])
    writer.writerow([date_string, model_id, hls_area, ls_area, latency])
    f.close()

In [ ]:
def print_results(filename):
    import pandas as pd
    with pd.option_context('display.float_format', '{:0.4f}'.format):
        csv_data = pd.read_csv(filename)
        accuracy_avg = csv_data.groupby('id').accuracy.mean()
        #csv_data = csv_data.join(accuracy_avg, on='id', rsuffix='_avg')
        nt_gev10_avg = csv_data.groupby('id').nt_gev10.mean()
        #csv_data = csv_data.join(nt_gev10_avg, on='id', rsuffix='_avg')        
        display(csv_data)
        
def print_hw_results(filename):
    import pandas as pd
    display_width = pd.get_option('display.width')
    pd.set_option('display.width', 1000)
    with pd.option_context('display.float_format', '{:0.4f}'.format):
        csv_data = pd.read_csv(filename)
        display(csv_data)
    pd.set_option('display.width', display_width)

In [ ]:
def print_avg_results(filename):
    import pandas as pd
    with pd.option_context('display.float_format', '{:0.4f}'.format):
        csv_data = pd.read_csv(filename)
        accuracy_avg = csv_data.groupby('id').accuracy.mean()
        nt_gev10_avg = csv_data.groupby('id').nt_gev10.mean()
        accuracy_std = csv_data.groupby('id').accuracy.std()
        nt_gev10_std = csv_data.groupby('id').nt_gev10.std()        
        csv_avg_std_data = pd.DataFrame(accuracy_avg)
        csv_avg_std_data = csv_avg_std_data.join(nt_gev10_avg, on='id')
        
        csv_data = pd.read_csv(filename, usecols=['id', 'accuracy'])
        csv_data_count = csv_data.groupby(['id']).count().rename(columns={'accuracy':'count'})
        
        csv_avg_std_data = csv_avg_std_data.join(csv_data_count, on='id')
        
        display(csv_avg_std_data)

In [ ]:
def plot_avg_results(filename, accuracy_th=0.76, tracks_th=0.9, id_ordering=None):
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    %matplotlib inline
    with pd.option_context('display.float_format', '{:0.4f}'.format):
        restore_figsize = plt.rcParams["figure.figsize"]
        plt.rcParams["figure.figsize"] = (10,10)
        csv_data = pd.read_csv(filename)
        accuracy_avg = csv_data.groupby('id').accuracy.mean()
        nt_gev10_avg = csv_data.groupby('id').nt_gev10.mean()
        
        df = pd.DataFrame({'accuracy': accuracy_avg, 'nt_gev10': nt_gev10_avg})
        #df = pd.DataFrame({'accuracy': accuracy_avg})
        if id_ordering != None:
            df = df.reindex(id_ordering)
        yticks=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        #ax = df.set_index('id').loc[id_ordering].plot.bar(rot=65, title='Avg metrics', ylim=[0.,1], yticks=yticks)
        ax = df.plot.barh(rot=0, title='Avg metrics', ylim=[0.,1], yticks=yticks)
        for container in ax.containers:
            ax.bar_label(container, fmt='%.4f', fontsize=8)
        
        # Shrink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])

        # Put a legend below current axis
        ax.legend(loc='upper right', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=5)
        
        # Threshold lines
        ax.axvline(tracks_th, color="orange", linestyle=":")
        ax.text(tracks_th, -2.0,
                str(tracks_th*100) + '%',
                va='center',
                ha='center',
                bbox=dict(facecolor="w",alpha=0.5, boxstyle='rarrow', edgecolor='orange'),
                #transform=ax.get_yaxis_transform(),
                rotation=90)
        
        ax.axvline(accuracy_th, color="lightblue", linestyle=":")
        ax.text(accuracy_th, -2.0,
                str(accuracy_th*100) + '%',
                va='center',
                ha='center',
                bbox=dict(facecolor="w",alpha=0.5, boxstyle='rarrow', edgecolor='lightblue'),
                #transform=ax.get_yaxis_transform(),
                rotation=90)
        
        plt.rcParams["figure.figsize"] = restore_figsize

In [ ]:
def print_dictionary(d, indent=0):
   for key, value in d.items():
      print('  ' * indent + str(key))
      if isinstance(value, dict):
         print_dictionary(value, indent+1)
      else:
         print('  ' * (indent+1) + str(value))

In [ ]:
def get_model_ids(lrange=range(12),
                  srange=['noscaling'],
                  mrange=['keras_d64', 'qkeras_foldbatchnorm_d64w6a10', 'hls4ml_qkeras_foldbatchnorm_d64w6a10'],
                  reverse=True
                 ):
    import itertools as it
    id_ordering = ['ds8l{}_{}_{}'.format(str(l), s, m) for [l, s, m] in it.product(
        lrange, # local bin
        srange, # dataset scaling
        mrange) # models
    ]
    if reverse:
        id_ordering.reverse()
    return id_ordering

## Prepare dataset

### Load dataset

#### ds7

`dataset 7`

#### ds-balanced

Balanced dataset (deprecated)

#### ds7q

`dataset 7` and the 14th column is distributed on the range `[0,11]`

#### ds678q

`dataset 6-7-8` (larger dataset) and the 14th column is distributed on the range `[0,11]`

#### ds8

`dataset 8`

#### ds8q

`dataset 8` and the 14th column is distributed on the range `[0,11]`

#### ds8-local

`dataset 8` but split for _each local bin_

<p style="background-color:Yellow;"><b>Choose the local bin (y-local).</b></p>

In [ ]:
local_id = 0 # 0 - 11

<p style="background-color:Yellow;"><b>Run identifier.</b></p>

In [ ]:
run_id = 0

In [ ]:
dataset = 'ds8l' + str(local_id) + '_'

base_dir = 'data/ds8_only'

train_data = base_dir + '/dec6_ds8_quant/QuantizedInputTrainSetLocal{}.csv'.format(local_id)
train_label = base_dir + '/dec6_ds8_quant/TrainSetLabelLocal{}.csv'.format(local_id)
test_data = base_dir + '/dec6_ds8_quant/QuantizedInputTestSetLocal{}.csv'.format(local_id)
test_label = base_dir + '/dec6_ds8_quant/TestSetLabelLocal{}.csv'.format(local_id)

pt_data = base_dir + '/dec6_ds8_quant/TestSetTruePTLocal{}.csv'.format(local_id)

df1 = pd.read_csv(train_data)
df2 = pd.read_csv(train_label)
df3 = pd.read_csv(test_data)
df4 = pd.read_csv(test_label)

In [ ]:
print(df1)

### Pad dataset

<p style="background-color:Tomato;"><b>DO NOT USE A PADDED DATASET TO TRAIN AND EVALUATE A MODEL! Padding a dataset is useful only to generate a "larger" accelerator in the synthesis flow.</b></p>

Enable following cell to enable padding.

In [ ]:
padding = ""

In [ ]:
padding = "padded_"

if '14' not in df1:
    df1['14'] = 0
df1['15'] = 0
df1['16'] = 0

if '14' not in df3:
    df3['14'] = 0
df3['15'] = 0
df3['16'] = 0

In [ ]:
X_train = df1.values
X_test = df3.values

y_train = df2.values
y_test = df4.values

print('Trainig set shape         :', X_train.shape) 
print('Trainig set shape (labels):', y_train.shape)
print('Test set shape:           :', X_test.shape)
print('Test set shape (labels)   :', y_test.shape)

### Visualize dataset

Observe the span of the value for each of the 14 columns. The 14th column -- if quantized -- should span on the range `[0,11]`.

In [ ]:
frames = [df1, df3]

df = pd.concat(frames)

In [ ]:
_ = plt.title("dataset with outliers")
_ = plt.boxplot(df.values, showfliers=True)

In [ ]:
_ = plt.title("dataset without outliers")
_ = plt.boxplot(df.values, showfliers=False)

You should see all integers if you are using the `dataset 6-7-8` with 14th column quantized

In [ ]:
pd.DataFrame(X_test[:5])

In [ ]:
import math
max_value_X_train = np.max(X_train)
min_value_X_train = np.min(X_train)
max_value_X_test = np.max(X_test)
min_value_X_test = np.min(X_test)

log2_max_value_X_train = int(np.ceil(math.log2(np.abs(max_value_X_train))))
#log2_min_value_X_train = int(np.ceil(math.log2(np.abs(min_value_X_train))))
log2_max_value_X_test = int(np.ceil(math.log2(np.abs(max_value_X_test))))
#log2_min_value_X_test = int(np.ceil(math.log2(np.abs(min_value_X_test))))

print('X_train: max=', max_value_X_train, ', log2(max)=', log2_max_value_X_train, ', min=', min_value_X_train)
print('X_test: max=', max_value_X_test, ', log2(max)=', log2_max_value_X_test, ', min=', min_value_X_test)

### Scale dataset

<p style="background-color:Yellow;"><b>Disabling scaling will simplify the hardware design</b></p>

In [ ]:
scale = False

In [ ]:
scaling = ''
if scale:
    scaling = 'scaling_'
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
else:
    scaling = 'noscaling_'

#### Visualize dataset post-scaling

In [ ]:
pd.DataFrame(X_train).to_csv('csv/' + dataset + scaling + 'InputTrainSetScale.csv',index=False)
pd.DataFrame(X_test).to_csv('csv/' + dataset + scaling + 'InputTestSetScale.csv',index=False)

In [ ]:
pd.DataFrame(X_test[:5])

In [ ]:
import math
max_value_X_train = np.max(X_train)
min_value_X_train = np.min(X_train)
max_value_X_test = np.max(X_test)
min_value_X_test = np.min(X_test)

log2_max_value_X_train = int(np.ceil(math.log2(np.abs(max_value_X_train))))
#log2_min_value_X_train = int(np.ceil(math.log2(np.abs(min_value_X_train))))
log2_max_value_X_test = int(np.ceil(math.log2(np.abs(max_value_X_test))))
#log2_min_value_X_test = int(np.ceil(math.log2(np.abs(min_value_X_test))))

print('X_train: max=', max_value_X_train, ', log2(max)=', log2_max_value_X_train, ', min=', min_value_X_train)
print('X_test: max=', max_value_X_test, ', log2(max)=', log2_max_value_X_test, ', min=', min_value_X_test)

### One-hot encoding

In [ ]:
y_train_oh = pd.get_dummies(df2['ptLabel'])
y_test_oh = pd.get_dummies(df4['ptLabel'])

y_train_oh.to_csv("csv/" + dataset + "labelsTrainOH.csv",index=False)
y_test_oh.to_csv("csv/" + dataset + "labelsTestOH.csv",index=False)

In [ ]:
display_side_by_side(pd.DataFrame(y_test[:5]), pd.DataFrame(y_test_oh[:5]), titles=['Prediction','One-hot encoding'])

### Save .dat files

## Model (Keras)

Let's start with a Keras model that is more traditional and uses floating-point. Our goal is to eventually get a quantized model for QKeras and hls4ml that is _close enough_ to this initial model.

### Model definition

Define a few MLP models with different size and number of layers

In [ ]:
models = {}

#### d128

This is our original model that we also use as a reference

In [ ]:
def CreateModel(shape, nb_classes):
    x = x_in = Input(shape, name="input")
    x = Dense(128, name="dense1")(x)
    x = keras.layers.BatchNormalization()(x)
    x = Activation("relu", name="relu1")(x)
    x = Dense(3, name="dense2")(x)
    x = Activation("linear", name="linear")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

models['d128'] = {
    'prefix': dataset + padding + scaling + 'keras_d128',
    'type'  : 'keras',
    'def'   : CreateModel}

#### d64

In [ ]:
def CreateModel(shape, nb_classes):
    x = x_in = Input(shape, name="input")
    x = Dense(64, name="dense1")(x)
    x = keras.layers.BatchNormalization()(x)
    x = Activation("relu", name="relu1")(x)
    x = Dense(3, name="dense2")(x)
    x = Activation("linear", name="linear")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

models['d64'] = {
    'prefix': dataset + padding + scaling + 'keras_d64',
    'type'  : 'keras',
    'def'   : CreateModel}

#### d60

In [ ]:
def CreateModel(shape, nb_classes):
    x = x_in = Input(shape, name="input")
    x = Dense(60, name="dense1")(x)
    x = keras.layers.BatchNormalization()(x)
    x = Activation("relu", name="relu1")(x)
    x = Dense(3, name="dense2")(x)
    x = Activation("linear", name="linear")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

models['d60'] = {
    'prefix': dataset + padding + scaling + 'keras_d60',
    'type'  : 'keras',
    'def'   : CreateModel}

#### d58

In [ ]:
def CreateModel(shape, nb_classes):
    x = x_in = Input(shape, name="input")
    x = Dense(58, name="dense1")(x)
    x = keras.layers.BatchNormalization()(x)
    x = Activation("relu", name="relu1")(x)
    x = Dense(3, name="dense2")(x)
    x = Activation("linear", name="linear")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

models['d58'] = {
    'prefix': dataset + padding + scaling + 'keras_d58',
    'type'  : 'keras',
    'def'   : CreateModel}

### Model create and summary

Show the dictionary of models

In [ ]:
print_dictionary(models)

<p style="background-color:Yellow;"><b>Choose a model for training and testing</b></p>


In [ ]:
#chosen_model = 'd64' # <<< PAY ATTENTION <<<
#chosen_model = 'd60' # <<< PAY ATTENTION <<<
chosen_model = 'd58' # <<< PAY ATTENTION <<<

In [ ]:
model_prefix = models[chosen_model]['prefix']
model_def = models[chosen_model]['def']
model_type = models[chosen_model]['type']

In [ ]:
print(model_prefix)

In [ ]:
print(y_train)

In [ ]:
model = model_def(X_train.shape[1:], y_train.shape[-1])

model.compile(optimizer=Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), # default from_logits=False
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

model.summary()

### Model training

In [ ]:
train_and_save = True # <<< PAY ATTENTION <<<

In [ ]:
model_file = 'models/' + model_prefix + 'model.h5'

history = None
if train_and_save:
    es = EarlyStopping(monitor='val_loss',
                       #monitor='val_sparse_categorical_accuracy', 
                       #mode='max', # don't minimize the accuracy!
                       patience=20,
                       restore_best_weights=True)

    history = model.fit(X_train,
                    y_train,
                    callbacks=[es],
                    epochs=150, 
                    batch_size=512,
                    validation_split=0.2,
                    shuffle=True,
                    verbose=0)
    
    model.save(model_file)
co = {}
utils._add_supported_quantized_objects(co)
model = tf.keras.models.load_model(model_file, custom_objects=co)

### Model evaluation

In [ ]:
if train_and_save:
    history_dict = history.history
    loss_values = history_dict['loss'] 
    val_loss_values = history_dict['val_loss'] 
    epochs = range(1, len(loss_values) + 1) 
    plt.plot(epochs, loss_values, 'bo', label='Training loss')
    plt.plot(epochs, val_loss_values, 'orange', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('images/' + model_prefix + '_loss.png')
else:
    from PIL import Image

    img = Image.open('images/' + model_prefix + '_loss.png')
    display(img)

In [ ]:
if train_and_save:
    acc = history.history['sparse_categorical_accuracy']
    val_acc = history.history['val_sparse_categorical_accuracy']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'orange', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    #np.max(val_acc)
    plt.savefig('images/' + model_prefix + '_accuracy.png')
    plt.show()
else:
    from PIL import Image
    #from os.path import exists

    img = Image.open('images/' + model_prefix + '_accuracy.png')
    display(img)

In [ ]:
preds = model.predict(X_test) 
predictionsFiles = np.argmax(preds, axis=1)

In [ ]:
pd.DataFrame(predictionsFiles).to_csv('csv/' + model_prefix + '_predictionsFiles.csv', header='predict', index=False)

In [ ]:
pd.DataFrame(y_test).to_csv('csv/' + model_prefix + '_true.csv', header='true', index=False)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss (Keras) {}: {}".format(chosen_model, score[0]))
print("Test accuracy (Keras) {}: {}".format(chosen_model, score[1]))

In [ ]:
from sklearn import datasets, svm, metrics
disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, predictionsFiles)
disp.figure_.suptitle("Multiclassifier Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")
plt.savefig('images/' + model_prefix + '_confusionMatrix.png')
plt.show()

Besides ML accuracy, we can evaluate the model with a metric that is closer to the physical application. We use _the number of tracks that are greater of 1.0 GeV and that are correctly classified as high pT_.

In [ ]:
#GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0 = get_number_of_tracks(dataset + scaling + model_type + '_' + chosen_model, pt_data)
GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0 = [0, 0, 0, 0]

In [ ]:
print('---')
print('Number of tracks greater than')
print('- 0.2GeV correctly classified as high pT:', GeV_0_2)
print('- 0.5GeV correctly classified as high pT:', GeV_0_5)
print('- 1.0GeV correctly classified as high pT:', GeV_1_0, '<<<')
print('- 2.0GeV correctly classified as high pT:', GeV_2_0)
print('---')

Summary of the results.

The model **id** is a label of the kind `dataset_scale_mltype_mlmodel`:
- `dataset` can be `ds7` (the original dataset) or `ds678` (the larger dataset with quantized 14th column)
- `scale` can be `scaling` (use standard scaler) or `noscaling` (don't scale)
- `mltype` can be `keras` (Keras), `qkeras` (QKeras), and `qkeras_foldbatchnorm` (QKeras + patch to fold batch normalization layer in the previous dense layer)
- `mlmodel` can be `d128`, `d64`, etc.

In [ ]:
with pd.option_context('display.float_format', '{:0.4f}'.format):
    data = pd.DataFrame([[model_prefix, score[0], score[1], GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0]], columns=["id", "loss", "accuracy", "nt_gev02", "nt_gev05", "nt_gev10", "nt_gev20"])
    display(data)

Current and previous results

In [ ]:
results_filename = 'csv/results.csv'
if train_and_save:
    write_results(results_filename, model_prefix, score[0], score[1], GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0)

In [ ]:
print_results(results_filename)

Average results so far

In [ ]:
print_avg_results(results_filename)

In [ ]:
id_ordering = get_model_ids(lrange=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
                            srange=['noscaling'],
                            mrange=[#'keras_d128',
                                    #'qkeras_foldbatchnorm_d128w5a10',
                                    #'hls4ml_qkeras_foldbatchnorm_d128w5a10',
                                    #
                                    #'keras_d64',
                                    #'qkeras_foldbatchnorm_d64w5a10',
                                    #'hls4ml_qkeras_foldbatchnorm_d64w5a10',
                                    #'qkeras_foldbatchnorm_d64w4a8',
                                    #'hls4ml_qkeras_foldbatchnorm_d64w4a8',
                                    #
                                    #'keras_d60',
                                    #'qkeras_foldbatchnorm_d60w5a10',
                                    #'hls4ml_qkeras_foldbatchnorm_d60w5a10',
                                    #'qkeras_foldbatchnorm_d60w4a8',
                                    #'hls4ml_qkeras_foldbatchnorm_d60w4a8',
                                    #
                                    'keras_d58',
                                    #'qkeras_foldbatchnorm_d58w5a10',
                                    #'hls4ml_qkeras_foldbatchnorm_d58w5a10',
                                    'qkeras_foldbatchnorm_d58w4a8',
                                    'hls4ml_qkeras_foldbatchnorm_d58w4a8',
                                   ],
                            reverse=True
                           )

In [ ]:
plot_avg_results(results_filename, accuracy_th=0.76, tracks_th=0.9, id_ordering=id_ordering)

In [ ]:
plot_pt_filter(local_id=local_id, 
               pt_data=pt_data,
               label_k='Keras y-local = {}'.format(local_id), predict_data_k='csv/' + model_prefix + '_predictionsFiles.csv', true_data_k='csv/' + model_prefix + '_true.csv',
               label_q=None, predict_data_q=None, true_data_q=None,
               label_h=None, predict_data_h=None, true_data_h=None,
               output_image='images/' + model_prefix + '_turnonCurve.png',
               base_dir='.')

- Dataset:
  - `ds7` is the `dataset 7`
  - `ds7q` is `dataset 7` where we also quantized 14th column on 12 bins
  - `ds8` is `dataset 8`
  - `ds8q` is `dataset 8` where we also quantized 14th column on 12 bins
  - `ds678q` is `dataset 6-7-8` where we also quantized 14th column on 12 bins
- All of the dataset are non scaled (`noscaling`)
- Model:
  - `keras` use Keras
  - `qkeras` use QKeras (quantized model)
- If it says `foldbatchnorm` then we folded the Batch Normalization layer in the previous Dense layer

## Model (QKeras)

Mostly the same as the previous section, but now we are usin QKeras

### Model definition

In [ ]:
qmodels = {}

#### qkeras-foldbatchnorm-d128w5a10

In [ ]:
# Fold BatchNormalization in QDense
def CreateQModel(shape, nb_classes):
    x = x_in = Input(shape, name="input1")
    x = QDenseBatchnorm(128,
        kernel_quantizer=quantized_bits(5,0,alpha=1),
        bias_quantizer=quantized_bits(5,0,alpha=1),
        name="dense1")(x)
    x = QActivation("quantized_relu(10,0)", name="relu1")(x)
    x = QDense(3,
        kernel_quantizer=quantized_bits(5,0,alpha=1),
        bias_quantizer=quantized_bits(5,0,alpha=1),
        name="dense2")(x)
    x = Activation("linear", name="linear")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

qmodels['d128w5a10'] = {
    'prefix': dataset + padding + scaling + 'qkeras_foldbatchnorm_d128w5a10',
    'type'  : 'qkeras_foldbatchnorm',
    'def'   : CreateQModel}

#### qkeras-foldbatchnorm-d128w4a8

In [ ]:
# Fold BatchNormalization in QDense
def CreateQModel(shape, nb_classes):
    x = x_in = Input(shape, name="input1")
    x = QDenseBatchnorm(128,
        kernel_quantizer=quantized_bits(4,0,alpha=1),
        bias_quantizer=quantized_bits(4,0,alpha=1),
        name="dense1")(x)
    x = QActivation("quantized_relu(8,0)", name="relu1")(x)
    x = QDense(3,
        kernel_quantizer=quantized_bits(4,0,alpha=1),
        bias_quantizer=quantized_bits(4,0,alpha=1),
        name="dense2")(x)
    x = Activation("linear", name="linear")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

qmodels['d128w4a8'] = {
    'prefix': dataset + padding + scaling + 'qkeras_foldbatchnorm_d128w4a8',
    'type'  : 'qkeras_foldbatchnorm',
    'def'   : CreateQModel}

#### qkeras-foldbatchnorm-d64w5a10

In [ ]:
# Fold BatchNormalization in QDense
def CreateQModel(shape, nb_classes):
    x = x_in = Input(shape, name="input1")
    x = QDenseBatchnorm(64,
        kernel_quantizer=quantized_bits(5,0,alpha=1),
        bias_quantizer=quantized_bits(5,0,alpha=1),
        name="dense1")(x)
    x = QActivation("quantized_relu(10,0)", name="relu1")(x)
    x = QDense(3,
        kernel_quantizer=quantized_bits(5,0,alpha=1),
        bias_quantizer=quantized_bits(5,0,alpha=1),
        name="dense2")(x)
    x = Activation("linear", name="linear")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

qmodels['d64w5a10'] = {
    'prefix': dataset + padding + scaling + 'qkeras_foldbatchnorm_d64w5a10',
    'type'  : 'qkeras_foldbatchnorm',
    'def'   : CreateQModel}

#### qkeras-foldbatchnorm-d64w4a8

In [ ]:
# Fold BatchNormalization in QDense
def CreateQModel(shape, nb_classes):
    x = x_in = Input(shape, name="input1")
    x = QDenseBatchnorm(64,
        kernel_quantizer=quantized_bits(4,0,alpha=1),
        bias_quantizer=quantized_bits(4,0,alpha=1),
        name="dense1")(x)
    x = QActivation("quantized_relu(8,0)", name="relu1")(x)
    x = QDense(3,
        kernel_quantizer=quantized_bits(4,0,alpha=1),
        bias_quantizer=quantized_bits(4,0,alpha=1),
        name="dense2")(x)
    x = Activation("linear", name="linear")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

qmodels['d64w4a8'] = {
    'prefix': dataset + padding + scaling + 'qkeras_foldbatchnorm_d64w4a8',
    'type'  : 'qkeras_foldbatchnorm',
    'def'   : CreateQModel}

#### qkeras-foldbatchnorm-d60w5a10

In [ ]:
# Fold BatchNormalization in QDense
def CreateQModel(shape, nb_classes):
    x = x_in = Input(shape, name="input1")
    x = QDenseBatchnorm(60,
        kernel_quantizer=quantized_bits(5,0,alpha=1),
        bias_quantizer=quantized_bits(5,0,alpha=1),
        name="dense1")(x)
    x = QActivation("quantized_relu(10,0)", name="relu1")(x)
    x = QDense(3,
        kernel_quantizer=quantized_bits(5,0,alpha=1),
        bias_quantizer=quantized_bits(5,0,alpha=1),
        name="dense2")(x)
    x = Activation("linear", name="linear")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

qmodels['d60w5a10'] = {
    'prefix': dataset + padding + scaling + 'qkeras_foldbatchnorm_d60w5a10',
    'type'  : 'qkeras_foldbatchnorm',
    'def'   : CreateQModel}

#### qkeras-foldbatchnorm-d60w4a8

In [ ]:
# Fold BatchNormalization in QDense
def CreateQModel(shape, nb_classes):
    x = x_in = Input(shape, name="input1")
    x = QDenseBatchnorm(60,
        kernel_quantizer=quantized_bits(4,0,alpha=1),
        bias_quantizer=quantized_bits(4,0,alpha=1),
        name="dense1")(x)
    x = QActivation("quantized_relu(8,0)", name="relu1")(x)
    x = QDense(3,
        kernel_quantizer=quantized_bits(4,0,alpha=1),
        bias_quantizer=quantized_bits(4,0,alpha=1),
        name="dense2")(x)
    x = Activation("linear", name="linear")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

qmodels['d60w4a8'] = {
    'prefix': dataset + padding + scaling + 'qkeras_foldbatchnorm_d60w4a8',
    'type'  : 'qkeras_foldbatchnorm',
    'def'   : CreateQModel}

#### qkeras-foldbatchnorm-d58w5a10

In [ ]:
# Fold BatchNormalization in QDense
def CreateQModel(shape, nb_classes):
    x = x_in = Input(shape, name="input1")
    x = QDenseBatchnorm(58,
        kernel_quantizer=quantized_bits(5,0,alpha=1),
        bias_quantizer=quantized_bits(5,0,alpha=1),
        name="dense1")(x)
    x = QActivation("quantized_relu(10,0)", name="relu1")(x)
    x = QDense(3,
        kernel_quantizer=quantized_bits(5,0,alpha=1),
        bias_quantizer=quantized_bits(5,0,alpha=1),
        name="dense2")(x)
    x = Activation("linear", name="linear")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

qmodels['d58w5a10'] = {
    'prefix': dataset + padding + scaling + 'qkeras_foldbatchnorm_d58w5a10',
    'type'  : 'qkeras_foldbatchnorm',
    'def'   : CreateQModel}

#### qkeras-foldbatchnorm-d58w4a8

In [ ]:
# Fold BatchNormalization in QDense
def CreateQModel(shape, nb_classes):
    x = x_in = Input(shape, name="input1")
    
    x = QDenseBatchnorm(58,
      kernel_quantizer=quantized_bits(4,0,alpha=1),
      bias_quantizer=quantized_bits(4,0,alpha=1),
      name="dense1")(x)
    # x = QDense(58,
    #      kernel_quantizer=quantized_bits(4,0,alpha=1),
    #      bias_quantizer=quantized_bits(4,0,alpha=1),
    #      name="dense1")(x)
    # x = keras.layers.BatchNormalization()(x)
    
    x = QActivation("quantized_relu(8,0)", name="relu1")(x)
    x = QDense(3,
        kernel_quantizer=quantized_bits(4,0,alpha=1),
        bias_quantizer=quantized_bits(4,0,alpha=1),
        name="dense2")(x)
    x = Activation("linear", name="linear")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

qmodels['d58w4a8'] = {
    'prefix': dataset + padding + scaling + 'qkeras_foldbatchnorm_d58w4a8',
    'type'  : 'qkeras_foldbatchnorm',
    'def'   : CreateQModel}

### Model create and summary

In [ ]:
print_dictionary(qmodels)

<p style="background-color:Yellow;"><b>Choose a QKeras model for training and testing. Pay attention to the previously created model in Keras.</b></p>

In [ ]:
print('Keras model:', chosen_model)

In [ ]:
#chosen_qmodel = 'd64w5a10'  # <<< PAY ATTENTION <<<
#chosen_qmodel = 'd64w4a8'  # <<< PAY ATTENTION <<<
#chosen_qmodel = 'd60w5a10'  # <<< PAY ATTENTION <<<
#chosen_qmodel = 'd60w4a8'  # <<< PAY ATTENTION <<<
#chosen_qmodel = 'd58w5a10'  # <<< PAY ATTENTION <<<
chosen_qmodel = 'd58w4a8'  # <<< PAY ATTENTION <<<

In [ ]:
qmodel_prefix = qmodels[chosen_qmodel]['prefix']
qmodel_def = qmodels[chosen_qmodel]['def']
qmodel_type = qmodels[chosen_qmodel]['type']

In [ ]:
model = qmodel_def(X_train.shape[1:], y_train.shape[-1])

model.compile(optimizer=Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), # default from_logits=False
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

model.summary()

### Model training

In [ ]:
train_and_save = True # <<< PAY ATTENTION <<<

In [ ]:
from qkeras.utils import model_save_quantized_weights

model_file = 'models/' + qmodel_prefix + 'model.h5'
model_q_weights_file = 'models/' + qmodel_prefix + 'model_q_weights.h5'

history = None
if train_and_save:
    es = EarlyStopping(monitor='val_loss',
                       #monitor='val_sparse_categorical_accuracy', 
                       #mode='max', # don't minimize the accuracy!
                       patience=20,
                       restore_best_weights=True)

    history = model.fit(X_train,
                    y_train,
                    callbacks=[es],
                    epochs=150, 
                    batch_size=1024,
                    validation_split=0.2,
                    shuffle=True,
                    verbose=0)
    
    model_save_quantized_weights(model, model_q_weights_file)
    model.save(model_file)
    print('Save:', model_file)
    print('Save:', model_q_weights_file)

co = {}
utils._add_supported_quantized_objects(co)
model = tf.keras.models.load_model(model_file, custom_objects=co)

### Model evaluation

In [ ]:
if train_and_save:
    history_dict = history.history
    loss_values = history_dict['loss'] 
    val_loss_values = history_dict['val_loss'] 
    epochs = range(1, len(loss_values) + 1) 
    plt.plot(epochs, loss_values, 'bo', label='Training loss')
    plt.plot(epochs, val_loss_values, 'orange', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('images/' + qmodel_prefix + '_loss.png')
else:
    from PIL import Image

    img = Image.open('images/' + qmodel_prefix + '_loss.png')
    display(img)

In [ ]:
if train_and_save:
    acc = history.history['sparse_categorical_accuracy']
    val_acc = history.history['val_sparse_categorical_accuracy']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'orange', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    #np.max(val_acc)
    plt.savefig('images/' + qmodel_prefix + '_accuracy.png')
    plt.show()
else:
    from PIL import Image
    #from os.path import exists

    img = Image.open('images/' + qmodel_prefix + '_accuracy.png')
    display(img)

In [ ]:
preds = model.predict(X_test) 
predictionsFiles = np.argmax(preds, axis=1)

In [ ]:
pd.DataFrame(predictionsFiles).to_csv("csv/" + qmodel_prefix + "_predictionsFiles.csv", header='predict', index=False)

In [ ]:
pd.DataFrame(y_test).to_csv("csv/" + qmodel_prefix + "_true.csv", header='true', index=False)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss (QKeras) {}: {}".format(chosen_model, score[0]))
print("Test accuracy (QKeras) {}: {}".format(chosen_model, score[1]))

In [ ]:
from sklearn import datasets, svm, metrics
disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, predictionsFiles)
disp.figure_.suptitle("Multiclassifier Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")
plt.savefig('images/' + qmodel_prefix + '_confusionMatrix.png')
plt.show()

In [ ]:
#GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0 = get_number_of_tracks(dataset + scaling + qmodel_type + '_' + chosen_qmodel, pt_data)
GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0 = [0, 0, 0, 0]

In [ ]:
print('---')
print('Number of tracks greater than')
print('- 0.2GeV correctly classified as high pT:', GeV_0_2)
print('- 0.5GeV correctly classified as high pT:', GeV_0_5)
print('- 1.0GeV correctly classified as high pT:', GeV_1_0)
print('- 2.0GeV correctly classified as high pT:', GeV_2_0)
print('---')

In [ ]:
with pd.option_context('display.float_format', '{:0.4f}'.format):
    data = pd.DataFrame([[qmodel_prefix, score[0], score[1], GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0]], columns=["id", "loss", "accuracy", "nt_gev02", "nt_gev05", "nt_gev10", "nt_gev20"])
    display(data)

In [ ]:
results_filename = 'csv/results.csv'
if train_and_save:
    write_results(results_filename, qmodel_prefix, score[0], score[1], GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0)

In [ ]:
print_results(results_filename)

In [ ]:
print_avg_results(results_filename)

In [ ]:
id_ordering = get_model_ids(lrange=range(12),
                            srange=['noscaling'],
                            mrange=[#'keras_d128',
                                    #'qkeras_foldbatchnorm_d128w5a10',
                                    #'hls4ml_qkeras_foldbatchnorm_d128w5a10',
                                    #
                                    #'keras_d64',
                                    #'qkeras_foldbatchnorm_d64w5a10',
                                    #'hls4ml_qkeras_foldbatchnorm_d64w5a10',
                                    #'qkeras_foldbatchnorm_d64w4a8',
                                    #'hls4ml_qkeras_foldbatchnorm_d64w4a8',
                                    #
                                    #'keras_d60',
                                    #'qkeras_foldbatchnorm_d60w5a10',
                                    #'hls4ml_qkeras_foldbatchnorm_d60w5a10',
                                    #'qkeras_foldbatchnorm_d60w4a8',
                                    #'hls4ml_qkeras_foldbatchnorm_d60w4a8',
                                    #
                                    #'keras_d58',
                                    #'qkeras_foldbatchnorm_d58w5a10',
                                    #'hls4ml_qkeras_foldbatchnorm_d58w5a10',
                                    'qkeras_foldbatchnorm_d58w4a8',
                                    'hls4ml_qkeras_foldbatchnorm_d58w4a8',
                                   ],
                            reverse=True
                           )

In [ ]:
plot_avg_results(results_filename, accuracy_th=0.76, tracks_th=0.9, id_ordering=id_ordering)

In [ ]:
plot_pt_filter(local_id=local_id, 
               pt_data=pt_data,
               label_k='Keras y-local = {}'.format(local_id), predict_data_k='csv/' + model_prefix + '_predictionsFiles.csv', true_data_k='csv/' + model_prefix + '_true.csv',
               label_q='QKeras y-local = {}'.format(local_id), predict_data_q='csv/' + qmodel_prefix + '_predictionsFiles.csv', true_data_q='csv/' + qmodel_prefix + '_true.csv',
               label_h=None, predict_data_h=None, true_data_h=None,
               output_image='images/' + model_prefix + '_turnonCurve.png',
               base_dir='.')

## Hardware translation

### hls4ml configuration

#### Setup

In [ ]:
import hls4ml
import plotting

hmodel_prefix = dataset + padding + scaling + 'hls4ml_' + qmodel_type + '_' + chosen_qmodel

In [ ]:
print('------------------------------------')
print(hmodel_prefix)
print('------------------------------------')

In [ ]:
print('hls4ml ver.', hls4ml.__version__)

In [ ]:
import os
os.environ['PATH'] = '/fpga/cad/xilinx/Vivado/2019.2/bin:' + os.environ['PATH']
def is_tool(name):
    from distutils.spawn import find_executable
    return find_executable(name) is not None

print('-----------------------------------')
if not is_tool('vivado_hls'):
    print('Xilinx Vivado HLS is NOT in the PATH')
else:
    print('Xilinx Vivado HLS is in the PATH')
print('-----------------------------------')

In [ ]:
import os

print('-----------------------------------')
if os.environ.get('MGC_HOME') is not None and is_tool('catapult'):
    print('Siemens Catapult HLS is configured')
else:
    print('Siemens Catapult HLS is NOT configured')
print('-----------------------------------')

In [ ]:
input_size = 6

In [ ]:
hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(
    layers=['Activation'],
    rounding_mode='AP_RND_CONV',
    saturation_mode='AP_SAT')

hconfig = hls4ml.utils.config_from_keras_model(model, granularity='name')

# programmable weights (weights on the interface)
hconfig["Model"]["BramFactor"] = 0

# set bitwidth of the input features
hconfig['LayerName']['input1']['Precision']['result'] = 'ufixed<' + str(input_size) + ',' + str(input_size) + '>'

# enable tracing
for layer in hconfig['LayerName'].keys():
    hconfig['LayerName'][layer]['Trace'] = True

# hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
# hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
# hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'
#hconfig['Model']['Precision'] = 'ap_fixed<16,8>'
# Dense
#hconfig['LayerName']['dense1']['accum_t'] = 'ap_fixed<8,5>'
#hconfig['LayerName']['dense1']['Precision']['result'] = 'ap_fixed<8,5>'
#hconfig['LayerName']['dense1']['Precision']['default'] = 'ap_fixed<8,5>'
# ReLU
#hconfig['LayerName']['relu1']['Precision']['result'] = 'ap_fixed<8,0,AC_RND_CONV,AC_SAT>'
# Dense
#hconfig['LayerName']['dense2']['accum_t'] = 'ap_fixed<10,6>'
#hconfig['LayerName']['dense2']['Precision']['result'] = 'ap_fixed<10,6>'
# SoftMax
#hconfig['LayerName']['softmax']['Precision'] = 'ap_fixed<128,64>'
#hconfig['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<18,8>'
#hconfig['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<18,4>'
#hconfig['LayerName']['softmax']['Precision'] = 'ap_fixed<16,6>'
# Required for the folding of BatchNormalization
#hconfig['SkipOptimizers'] = ['propagate_dense_precision']    
#hconfig['LayerName']['dense1_linear']['Trace'] = True
#hconfig['LayerName']['dense2_linear']['Trace'] = True

#### Convert

In [ ]:
plotting.print_dict(hconfig)

In [ ]:
!rm -rf $hmodel_prefix\_$run_id\_catapult_prj*

In [ ]:
hmodel = hls4ml.converters.convert_from_keras_model(model,
                                                    clock_period=10.0,
                                                    hls_config=hconfig,
                                                    output_dir=hmodel_prefix + '_' + str(run_id) + '_catapult_prj',
                                                    backend='Catapult',
                                                    #output_dir=hmodel_prefix + '_' + str(run_id) + '_vivado_prj',
                                                    #backend='Vivado',
                                                    part='XC7A100T')

In [ ]:
hls4ml.utils.plot_model(hmodel, show_shapes=True, show_precision=True, to_file='model.png')

### Bit-accurate simulation

In [ ]:
hpreds, htrace = hmodel.trace(np.ascontiguousarray(X_test.astype(float)))

In [ ]:
trace = hls4ml.model.profiling.get_ymodel_keras(model, X_test)

In [ ]:
for layer in htrace.keys():
    plt.figure()
    klayer = layer
    if '_alpha' in layer:
        klayer = layer.replace('_alpha', '')
    if 'dense1_linear' in layer:
        continue
    if 'dense2_linear' in layer:
        continue
    plt.scatter(htrace[layer].flatten(), trace[klayer].flatten(), s=0.2)
    min_x = min(np.amin(htrace[layer]), np.amin(trace[klayer]))
    max_x = max(np.amax(htrace[layer]), np.amax(trace[klayer]))
    plt.plot([min_x, max_x], [min_x, max_x], c='gray')
    plt.xlabel('hls4ml {}'.format(layer))
    plt.ylabel('QKeras {}'.format(klayer))
    plt.savefig(os.path.join(hmodel_prefix + '_' + str(run_id) + '_catapult_prj', 'profiling_{}.png'.format(layer)), dpi=300)

### Model evaluation

In [ ]:
for layer in hconfig['LayerName'].keys():
    hconfig['LayerName'][layer]['Trace'] = False

hmodel = hls4ml.converters.convert_from_keras_model(model,
                                                    clock_period=10.0,
                                                    hls_config=hconfig,
                                                    output_dir=hmodel_prefix + '_' + str(run_id) + '_catapult_prj',
                                                    backend='Catapult',
                                                    #output_dir=hmodel_prefix + '_' + str(run_id) + '_vivado_prj',
                                                    #backend='Vivado',
                                                    part='XC7A100T')

hmodel.compile()

In [ ]:
hpreds = hmodel.predict(np.ascontiguousarray(X_test.astype(float)))
np.savetxt('tb_output_predictions_hw.dat', hpreds, fmt='%f')

predictionsFiles = np.argmax(hpreds, axis=1)
pd.DataFrame(predictionsFiles).to_csv("csv/" + hmodel_prefix + "_predictionsFiles.csv", header='predict', index=False)
pd.DataFrame(y_test).to_csv('csv/' + hmodel_prefix + '_true.csv', header='true', index=False)

In [ ]:
from sklearn.metrics import accuracy_score
print('-----------------------------------')
print("QKeras accuracy: {:.4f}%".format(100*accuracy_score(np.argmax(y_test_oh.values, axis=1), np.argmax(preds, axis=1))))
print("hls4ml accuracy: {:.4f}%".format(100*accuracy_score(np.argmax(y_test_oh.values, axis=1), np.argmax(hpreds, axis=1))))
print('-----------------------------------')

In [ ]:
hls4ml_accuracy = accuracy_score(np.argmax(y_test_oh.values, axis=1), np.argmax(hpreds, axis=1))

In [ ]:
GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0 = get_number_of_tracks(dataset + padding + scaling + 'hls4ml_' + qmodel_type + '_' + chosen_qmodel, pt_data)

In [ ]:
with pd.option_context('display.float_format', '{:0.4f}'.format):
    data = pd.DataFrame([[hmodel_prefix, 0, hls4ml_accuracy, GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0]], columns=["id", "loss", "accuracy", "nt_gev02", "nt_gev05", "nt_gev10", "nt_gev20"])
    display(data)

In [ ]:
results_filename = 'csv/results.csv'
write_results(results_filename, hmodel_prefix, 0, hls4ml_accuracy, GeV_0_2, GeV_0_5, GeV_1_0, GeV_2_0)

In [ ]:
print_results(results_filename)

In [ ]:
id_ordering = get_model_ids(lrange=range(12),
                            srange=['noscaling'],
                            mrange=[#'keras_d128',
                                    #'qkeras_foldbatchnorm_d128w5a10',
                                    #'hls4ml_qkeras_foldbatchnorm_d128w5a10',
                                    #
                                    #'keras_d64',
                                    #'qkeras_foldbatchnorm_d64w5a10',
                                    #'hls4ml_qkeras_foldbatchnorm_d64w5a10',
                                    #'qkeras_foldbatchnorm_d64w4a8',
                                    #'hls4ml_qkeras_foldbatchnorm_d64w4a8',
                                    #
                                    #'keras_d60',
                                    #'qkeras_foldbatchnorm_d60w5a10',
                                    #'hls4ml_qkeras_foldbatchnorm_d60w5a10',
                                    #'qkeras_foldbatchnorm_d60w4a8',
                                    #'hls4ml_qkeras_foldbatchnorm_d60w4a8',
                                    #
                                    'keras_d58',
                                    #'qkeras_foldbatchnorm_d58w5a10',
                                    #'hls4ml_qkeras_foldbatchnorm_d58w5a10',
                                    'qkeras_foldbatchnorm_d58w4a8',
                                    'hls4ml_qkeras_foldbatchnorm_d58w4a8',
                                   ],
                            reverse=True
                           )

In [ ]:
plot_avg_results(results_filename, accuracy_th=0.76, tracks_th=0.9, id_ordering=id_ordering)

In [ ]:
plot_pt_filter(local_id=local_id, 
               pt_data=pt_data,
               label_k='Keras y-local = {}'.format(local_id), predict_data_k='csv/' + model_prefix + '_predictionsFiles.csv', true_data_k='csv/' + model_prefix + '_true.csv',
               label_q='QKeras y-local = {}'.format(local_id), predict_data_q='csv/' + qmodel_prefix + '_predictionsFiles.csv', true_data_q='csv/' + qmodel_prefix + '_true.csv',
               label_h='hls4ml y-local = {}'.format(local_id), predict_data_h='csv/' + hmodel_prefix + '_predictionsFiles.csv', true_data_h='csv/' + hmodel_prefix + '_true.csv',
               output_image='images/' + hmodel_prefix + '_turnonCurve.png',
               base_dir='.')

### Synthesis

In [ ]:
%%time
results = hmodel.build(csim=False, synth=True, vsynth=True)

In [ ]:
def get_catapult_results(project_dir, project_name = 'myproject', solution_id = 'v1'):

    import re
    import sys

    hls_result = {'HLSArea': 0, 'LSArea': 0, 'Latency': 0}

    file = open('{}/{}_prj/{}.{}/rtl.rpt'.format(project_dir, project_name, project_name, solution_id), 'r')
    for line in file:
        if re.search('Total Area Score', line):
            hls_result['HLSArea'] = float(line.split()[5])
    file.close()

    file = open('{}/{}_prj/{}.{}/rtl.rpt'.format(project_dir, project_name, project_name, solution_id), 'r')
    for line in file:
        if re.search('Design Total:', line):
            hls_result['Latency'] = int(line.split()[3])
    file.close()
    
    file = open('{}/{}_prj/{}.{}/oasys.log.00'.format(project_dir, project_name, project_name, solution_id), 'r')
    for line in file:
        if re.search('Total Cell Area is', line):
            hls_result['LSArea'] = float(line.split()[4])
    file.close()
    
    return hls_result

In [ ]:
hls4ml_project_dir = hmodel_prefix + '_' + str(run_id) + '_catapult_prj'
hls_results = get_catapult_results(project_dir = hls4ml_project_dir)
print('HLS Area:', hls_results['HLSArea'])
print('LS Area:', hls_results['LSArea'])
print('Latency:', hls_results['Latency'])

In [ ]:
results_filename = 'csv/hw_results.csv'
write_hw_results(results_filename, hmodel_prefix, hls_results['HLSArea'], hls_results['LSArea'], hls_results['Latency'])

In [ ]:
def print_hw_results(filename):
    import pandas as pd
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.max_colwidth', None)
    with pd.option_context('display.float_format', '{:0.4f}'.format):
        csv_data = pd.read_csv(filename)
        display(csv_data)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.max_colwidth')

print_hw_results(results_filename)